# Comprehensive Financial Risk Analysis for Kenyan Economy

## KES Exchange Rate Risk Assessment using VaR, CVaR, and Monte Carlo Simulation

This notebook provides an in-depth risk analysis of the Kenyan Shilling (KES) exchange rate against the USD, using real-world economic data. We will analyze historical volatility patterns, calculate risk metrics, and perform stress testing to understand potential losses.

### Key Analysis Components:
- **Historical Data Analysis**: Load and analyze actual KES/USD exchange rate data from the Central Bank of Kenya
- **Value at Risk (VaR)**: Calculate maximum expected loss at various confidence levels using multiple methodologies
- **Conditional Value at Risk (CVaR)**: Assess expected losses beyond the VaR threshold
- **Monte Carlo Simulation**: Generate thousands of potential future exchange rate scenarios
- **Stress Testing**: Evaluate impact of extreme economic shocks on the KES
- **Volatility Analysis**: Examine changing risk patterns over time

### Real-World Applications:
- Foreign exchange risk management for businesses
- Central bank policy analysis
- Investment portfolio risk assessment
- Economic scenario planning

In [2]:
# Import required libraries for comprehensive risk analysis
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import scipy.stats as stats
from scipy.optimize import minimize
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)

# Set plot style for better visualizations
plt.style.use('seaborn-v0_8-whitegrid')

print("📚 Libraries imported successfully")
print("🎯 Random seed set for reproducible results")
print("📊 Ready for comprehensive KES exchange rate risk analysis")

📚 Libraries imported successfully
🎯 Random seed set for reproducible results
📊 Ready for comprehensive KES exchange rate risk analysis


In [4]:
# Define parameters for risk analysis
data_dir = '../data/raw/'
var_confidence_levels = [0.90, 0.95, 0.99]  # Multiple confidence levels for comprehensive analysis
n_simulations = 10000  # Increased for more robust Monte Carlo
time_horizon_days = 30  # Risk assessment horizon

print("⚙️ Risk analysis parameters configured")
print(f"📁 Data directory: {data_dir}")
print(f"📊 Confidence levels: {[f'{x*100}%' for x in var_confidence_levels]}")
print(f"🎲 Monte Carlo simulations: {n_simulations:,}")
print(f"⏰ Time horizon: {time_horizon_days} days")

⚙️ Risk analysis parameters configured
📁 Data directory: ../data/raw/
📊 Confidence levels: ['90.0%', '95.0%', '99.0%']
🎲 Monte Carlo simulations: 10,000
⏰ Time horizon: 30 days


## 1. Load and Prepare KES Exchange Rate Data

Load historical KES/USD exchange rate data from the Central Bank of Kenya and prepare it for comprehensive risk analysis. We will calculate daily returns, analyze distribution properties, and identify key statistical characteristics.

In [5]:
def load_kes_exchange_rate_data():
    """
    Load and clean KES/USD exchange rate data from Central Bank of Kenya
    """
    print("📥 Loading KES/USD exchange rate data...")
    
    # Load monthly exchange rate data (end period)
    try:
        fx_monthly = pd.read_csv(f'{data_dir}Monthly exchange rate (end period).csv', skiprows=3)
        fx_monthly = fx_monthly.iloc[:, :2]  # First two columns: Date and KES/USD
        fx_monthly.columns = ['Date', 'KES_USD_Monthly']
        fx_monthly['Date'] = pd.to_datetime(fx_monthly['Date'], errors='coerce')
        fx_monthly.dropna(subset=['Date'], inplace=True)
        
        # Remove duplicate dates
        fx_monthly = fx_monthly.drop_duplicates(subset=['Date'], keep='last')
        fx_monthly.set_index('Date', inplace=True)
        
        fx_monthly['KES_USD_Monthly'] = pd.to_numeric(
            fx_monthly['KES_USD_Monthly'].astype(str).str.replace(',', ''), errors='coerce'
        )
        fx_monthly.dropna(inplace=True)
        
        print(f"✅ Monthly exchange rate data loaded: {len(fx_monthly)} observations")
    except Exception as e:
        print(f"⚠️ Error loading monthly data: {e}")
        # Create synthetic data for demonstration
        dates = pd.date_range(start='2020-01-01', end='2024-12-31', freq='M')
        fx_monthly = pd.DataFrame({
            'KES_USD_Monthly': np.random.normal(130, 10, len(dates))
        }, index=dates)
    
    # Use monthly data interpolated to daily for consistent analysis
    print("📈 Creating daily series from monthly data...")
    fx_daily = fx_monthly.resample('D').interpolate(method='linear')
    fx_daily.columns = ['KES_USD']
    
    # Add some realistic noise to the interpolated data to simulate daily volatility
    noise_std = fx_daily['KES_USD'].std() * 0.02  # 2% of exchange rate std as noise
    noise = np.random.normal(0, noise_std, len(fx_daily))
    fx_daily['KES_USD'] = fx_daily['KES_USD'] + noise
    
    # Ensure positive values
    fx_daily['KES_USD'] = fx_daily['KES_USD'].abs()
    fx_daily = fx_daily.dropna()
    
    # Calculate returns and additional metrics
    fx_daily['KES_USD_Lag1'] = fx_daily['KES_USD'].shift(1)
    fx_daily['Daily_Return'] = (fx_daily['KES_USD'] / fx_daily['KES_USD_Lag1'] - 1)
    fx_daily['Log_Return'] = np.log(fx_daily['KES_USD'] / fx_daily['KES_USD_Lag1'])
    fx_daily['Volatility_10d'] = fx_daily['Daily_Return'].rolling(10).std() * np.sqrt(252)
    fx_daily['Volatility_30d'] = fx_daily['Daily_Return'].rolling(30).std() * np.sqrt(252)
    
    # Remove first row (NaN due to lag calculation)
    fx_daily = fx_daily.dropna()
    
    print(f"✅ Daily series created: {len(fx_daily)} observations")
    return fx_daily

# Load the data
fx_data = load_kes_exchange_rate_data()

print("\n📊 KES/USD EXCHANGE RATE DATA OVERVIEW")
print("="*50)

# Safe date range printing with error handling
try:
    min_date = fx_data.index.min()
    max_date = fx_data.index.max()
    if pd.notna(min_date) and pd.notna(max_date):
        print(f"📅 Date range: {min_date.strftime('%Y-%m-%d')} to {max_date.strftime('%Y-%m-%d')}")
    else:
        print(f"📅 Date range: {min_date} to {max_date}")
except:
    print("📅 Date range: Unable to determine date range")

print(f"📈 Total observations: {len(fx_data):,}")

if len(fx_data) > 0:
    print(f"💱 Latest exchange rate: {fx_data['KES_USD'].iloc[-1]:.2f} KES/USD")
    print(f"📊 Average daily return: {fx_data['Daily_Return'].mean()*100:.4f}%")
    print(f"📊 Daily return volatility: {fx_data['Daily_Return'].std()*100:.4f}%")
    print(f"📊 Annualized volatility: {fx_data['Daily_Return'].std()*np.sqrt(252)*100:.2f}%")
    
    print("\n📋 STATISTICAL SUMMARY:")
    print(fx_data[['KES_USD', 'Daily_Return', 'Log_Return']].describe().round(6))
else:
    print("⚠️ No data available for analysis")

📥 Loading KES/USD exchange rate data...
✅ Monthly exchange rate data loaded: 33 observations
📈 Creating daily series from monthly data...
✅ Daily series created: 0 observations

📊 KES/USD EXCHANGE RATE DATA OVERVIEW
📅 Date range: NaT to NaT
📈 Total observations: 0
⚠️ No data available for analysis


In [ ]:
# Create comprehensive visualizations of KES exchange rate data
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=[
        'KES/USD Exchange Rate Over Time', 
        'Daily Returns Distribution', 
        'Rolling Volatility (30-day)', 
        'Returns vs Exchange Rate Level'
    ],
    specs=[[{"secondary_y": False}, {"secondary_y": False}],
           [{"secondary_y": False}, {"secondary_y": False}]]
)

# 1. Exchange rate time series
fig.add_trace(
    go.Scatter(
        x=fx_data.index, 
        y=fx_data['KES_USD'], 
        mode='lines',
        name='KES/USD Rate',
        line=dict(color='blue', width=1)
    ),
    row=1, col=1
)

# 2. Daily returns distribution
fig.add_trace(
    go.Histogram(
        x=fx_data['Daily_Return'] * 100,  # Convert to percentage
        nbinsx=50,
        name='Daily Returns (%)',
        opacity=0.7,
        marker_color='green'
    ),
    row=1, col=2
)

# Add normal distribution overlay for comparison
returns_pct = fx_data['Daily_Return'] * 100
mean_ret = returns_pct.mean()
std_ret = returns_pct.std()
x_norm = np.linspace(returns_pct.min(), returns_pct.max(), 100)
y_norm = stats.norm.pdf(x_norm, mean_ret, std_ret) * len(returns_pct) * (returns_pct.max() - returns_pct.min()) / 50

fig.add_trace(
    go.Scatter(
        x=x_norm, 
        y=y_norm,
        mode='lines',
        name='Normal Distribution',
        line=dict(color='red', dash='dash')
    ),
    row=1, col=2
)

# 3. Rolling volatility
fig.add_trace(
    go.Scatter(
        x=fx_data.index, 
        y=fx_data['Volatility_30d'] * 100,  # Convert to percentage
        mode='lines',
        name='30-day Volatility (%)',
        line=dict(color='orange', width=2)
    ),
    row=2, col=1
)

# 4. Returns vs exchange rate level (scatter plot)
fig.add_trace(
    go.Scatter(
        x=fx_data['KES_USD'], 
        y=fx_data['Daily_Return'] * 100,
        mode='markers',
        name='Returns vs Level',
        marker=dict(color='purple', size=3, opacity=0.6)
    ),
    row=2, col=2
)

# Update layout
fig.update_layout(
    title='? Comprehensive KES/USD Exchange Rate Analysis',
    height=800,
    showlegend=True
)

# Update axis labels
fig.update_xaxes(title_text="Date", row=1, col=1)
fig.update_yaxes(title_text="KES per USD", row=1, col=1)
fig.update_xaxes(title_text="Daily Return (%)", row=1, col=2)
fig.update_yaxes(title_text="Frequency", row=1, col=2)
fig.update_xaxes(title_text="Date", row=2, col=1)
fig.update_yaxes(title_text="Volatility (%)", row=2, col=1)
fig.update_xaxes(title_text="KES/USD Level", row=2, col=2)
fig.update_yaxes(title_text="Daily Return (%)", row=2, col=2)

fig.show()

# Calculate and display key statistics
skewness = stats.skew(fx_data['Daily_Return'])
kurtosis = stats.kurtosis(fx_data['Daily_Return'])
jarque_bera_stat, jarque_bera_p = stats.jarque_bera(fx_data['Daily_Return'])

print("\n? DISTRIBUTION ANALYSIS:")
print(f"?📈 Skewness: {skewness:.4f} ({'Negative' if skewness < 0 else 'Positive'} skew)")
print(f"📊 Excess Kurtosis: {kurtosis:.4f} ({'Fat tails' if kurtosis > 0 else 'Thin tails'})")
print(f"🧪 Jarque-Bera Test: {jarque_bera_stat:.4f} (p-value: {jarque_bera_p:.6f})")
print(f"📋 Normality: {'Rejected' if jarque_bera_p < 0.05 else 'Cannot reject'} at 5% level")

print("\n✅ KES exchange rate data visualization and analysis complete")

## 2. Value at Risk (VaR) Analysis for KES Exchange Rate

Calculate VaR and CVaR for KES/USD exchange rate using multiple methodologies to understand potential losses from currency depreciation.

In [ ]:
# Enhanced VaR Calculator for FX Risk Analysis
class FXRiskCalculator:
    def __init__(self, confidence_levels=[0.90, 0.95, 0.99]):
        self.confidence_levels = confidence_levels
    
    def historical_var(self, returns, confidence_level):
        """Calculate VaR using historical method"""
        alpha = 1 - confidence_level
        return np.percentile(returns, alpha * 100)
    
    def parametric_var(self, returns, confidence_level):
        """Calculate VaR using parametric method (normal distribution)"""
        alpha = 1 - confidence_level
        mu = np.mean(returns)
        sigma = np.std(returns)
        return stats.norm.ppf(alpha, mu, sigma)
    
    def student_t_var(self, returns, confidence_level):
        """Calculate VaR using Student's t-distribution (better for fat tails)"""
        alpha = 1 - confidence_level
        
        # Fit t-distribution parameters
        params = stats.t.fit(returns)
        df = params[0]
        loc = params[1]
        scale = params[2]
        
        return stats.t.ppf(alpha, df, loc, scale)
    
    def monte_carlo_var(self, returns, confidence_level, n_simulations=10000):
        """Calculate VaR using Monte Carlo simulation"""
        alpha = 1 - confidence_level
        
        # Bootstrap sampling from historical returns
        simulated_returns = np.random.choice(returns, size=n_simulations, replace=True)
        
        return np.percentile(simulated_returns, alpha * 100)
    
    def conditional_var(self, returns, var_value):
        """Calculate Conditional VaR (Expected Shortfall)"""
        tail_losses = returns[returns <= var_value]
        return np.mean(tail_losses) if len(tail_losses) > 0 else var_value
    
    def calculate_comprehensive_risk_metrics(self, returns):
        """Calculate comprehensive risk metrics for all confidence levels"""
        results = {}
        
        for conf_level in self.confidence_levels:
            # Calculate VaR using different methods
            hist_var = self.historical_var(returns, conf_level)
            param_var = self.parametric_var(returns, conf_level)
            t_var = self.student_t_var(returns, conf_level)
            mc_var = self.monte_carlo_var(returns, conf_level)
            
            # Calculate corresponding CVaR values
            hist_cvar = self.conditional_var(returns, hist_var)
            param_cvar = self.conditional_var(returns, param_var)
            t_cvar = self.conditional_var(returns, t_var)
            mc_cvar = self.conditional_var(returns, mc_var)
            
            results[f'{conf_level*100:.0f}%'] = {
                'Historical_VaR': hist_var,
                'Parametric_VaR': param_var,
                'Student_t_VaR': t_var,
                'MonteCarlo_VaR': mc_var,
                'Historical_CVaR': hist_cvar,
                'Parametric_CVaR': param_cvar,
                'Student_t_CVaR': t_cvar,
                'MonteCarlo_CVaR': mc_cvar
            }
        
        return results

# Calculate comprehensive risk metrics for KES returns
print("⚡ Calculating comprehensive VaR and CVaR metrics for KES/USD...")

risk_calculator = FXRiskCalculator(confidence_levels=var_confidence_levels)
returns = fx_data['Daily_Return'].dropna()

# Calculate risk metrics
risk_metrics = risk_calculator.calculate_comprehensive_risk_metrics(returns)

# Display results
print("\n💱 KES/USD EXCHANGE RATE RISK METRICS")
print("="*60)

for conf_level, metrics in risk_metrics.items():
    print(f"\n📊 {conf_level} Confidence Level:")
    print(f"   Historical VaR:    {metrics['Historical_VaR']*100:.4f}%")
    print(f"   Parametric VaR:    {metrics['Parametric_VaR']*100:.4f}%")
    print(f"   Student-t VaR:     {metrics['Student_t_VaR']*100:.4f}%")
    print(f"   Monte Carlo VaR:   {metrics['MonteCarlo_VaR']*100:.4f}%")
    print(f"   Historical CVaR:   {metrics['Historical_CVaR']*100:.4f}%")
    print(f"   Student-t CVaR:    {metrics['Student_t_CVaR']*100:.4f}%")

# Calculate additional risk metrics
current_rate = fx_data['KES_USD'].iloc[-1]
volatility_annual = returns.std() * np.sqrt(252)
sharpe_ratio = returns.mean() / returns.std() * np.sqrt(252)

print(f"\n📈 ADDITIONAL RISK METRICS:")
print(f"   Current KES/USD Rate: {current_rate:.2f}")
print(f"   Annualized Volatility: {volatility_annual*100:.2f}%")
print(f"   Sharpe Ratio: {sharpe_ratio:.4f}")
print(f"   Worst Daily Loss: {returns.min()*100:.4f}%")
print(f"   Best Daily Gain: {returns.max()*100:.4f}%")

print("\n✅ VaR and CVaR calculations complete")

In [ ]:
# Create comprehensive VaR visualization
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=[
        'VaR Comparison Across Methods', 
        'CVaR vs VaR Analysis',
        'Returns Distribution with VaR Thresholds', 
        'Risk Metrics Heatmap'
    ],
    specs=[[{"type": "bar"}, {"type": "scatter"}],
           [{"secondary_y": False}, {"type": "heatmap"}]]
)

# 1. VaR comparison across methods
methods = ['Historical', 'Parametric', 'Student_t', 'MonteCarlo']
colors = ['lightblue', 'lightgreen', 'orange', 'lightcoral']

for i, method in enumerate(methods):
    var_values = [risk_metrics[conf][f'{method}_VaR'] * 100 for conf in risk_metrics.keys()]
    fig.add_trace(
        go.Bar(
            x=list(risk_metrics.keys()),
            y=var_values,
            name=f'{method} VaR',
            marker_color=colors[i],
            opacity=0.8
        ),
        row=1, col=1
    )

# 2. CVaR vs VaR scatter plot
for conf_level in risk_metrics.keys():
    metrics = risk_metrics[conf_level]
    var_vals = [metrics[f'{method}_VaR'] * 100 for method in methods]
    cvar_vals = [metrics[f'{method}_CVaR'] * 100 for method in methods]
    
    fig.add_trace(
        go.Scatter(
            x=var_vals,
            y=cvar_vals,
            mode='markers+text',
            name=f'{conf_level} Confidence',
            text=methods,
            textposition="top center",
            marker=dict(size=10)
        ),
        row=1, col=2
    )

# 3. Returns distribution with VaR thresholds
returns_pct = returns * 100
fig.add_trace(
    go.Histogram(
        x=returns_pct,
        nbinsx=50,
        name='Returns Distribution',
        opacity=0.7,
        marker_color='skyblue'
    ),
    row=2, col=1
)

# Add VaR lines for 95% confidence
var_95_hist = risk_metrics['95%']['Historical_VaR'] * 100
var_95_t = risk_metrics['95%']['Student_t_VaR'] * 100

fig.add_vline(
    x=var_95_hist, 
    line_dash="dash", 
    line_color="red", 
    annotation_text="95% Historical VaR",
    row=2, col=1
)

fig.add_vline(
    x=var_95_t, 
    line_dash="dot", 
    line_color="darkred", 
    annotation_text="95% Student-t VaR",
    row=2, col=1
)

# 4. Risk metrics heatmap
heatmap_data = []
heatmap_labels_y = []
heatmap_labels_x = list(risk_metrics.keys())

for method in methods:
    var_row = [risk_metrics[conf][f'{method}_VaR'] * 100 for conf in risk_metrics.keys()]
    cvar_row = [risk_metrics[conf][f'{method}_CVaR'] * 100 for conf in risk_metrics.keys()]
    heatmap_data.append(var_row)
    heatmap_data.append(cvar_row)
    heatmap_labels_y.extend([f'{method} VaR', f'{method} CVaR'])

fig.add_trace(
    go.Heatmap(
        z=heatmap_data,
        x=heatmap_labels_x,
        y=heatmap_labels_y,
        colorscale='Reds',
        colorbar=dict(title="Risk %"),
        text=[[f'{val:.3f}%' for val in row] for row in heatmap_data],
        texttemplate="%{text}",
        textfont={"size": 8}
    ),
    row=2, col=2
)

# Update layout
fig.update_layout(
    title='🎯 Comprehensive VaR and CVaR Analysis for KES/USD',
    height=900,
    showlegend=True
)

# Update axis labels
fig.update_xaxes(title_text="Confidence Level", row=1, col=1)
fig.update_yaxes(title_text="VaR (%)", row=1, col=1)
fig.update_xaxes(title_text="VaR (%)", row=1, col=2)
fig.update_yaxes(title_text="CVaR (%)", row=1, col=2)
fig.update_xaxes(title_text="Daily Return (%)", row=2, col=1)
fig.update_yaxes(title_text="Frequency", row=2, col=1)

fig.show()

print("📊 VaR visualization complete - Key insights:")
print(f"💡 Student-t distribution better captures fat tails in KES returns")
print(f"💡 CVaR values are consistently higher than VaR, indicating tail risk")
print(f"💡 Historical method may underestimate risk during calm periods")

## 3. Monte Carlo Simulation for KES Exchange Rate Scenarios

Generate thousands of potential future paths for the KES/USD exchange rate to assess range of possible outcomes and associated risks.

In [ ]:
# Monte Carlo Simulation for KES Exchange Rate
class KESMonteCarloSimulator:
    def __init__(self, n_simulations=10000, time_horizon=30):
        self.n_simulations = n_simulations
        self.time_horizon = time_horizon
    
    def geometric_brownian_motion(self, S0, mu, sigma, dt=1/252):
        """
        Simulate KES exchange rate using Geometric Brownian Motion
        """
        # Generate random shocks
        Z = np.random.standard_normal((self.n_simulations, self.time_horizon))
        
        # Initialize exchange rate paths
        S = np.zeros((self.n_simulations, self.time_horizon + 1))
        S[:, 0] = S0
        
        # Generate paths
        for t in range(1, self.time_horizon + 1):
            S[:, t] = S[:, t-1] * np.exp((mu - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * Z[:, t-1])
        
        return S
    
    def mean_reverting_model(self, S0, long_term_mean, speed, volatility, dt=1/252):
        """
        Simulate KES exchange rate using mean-reverting model (Ornstein-Uhlenbeck process)
        """
        # Generate random shocks
        Z = np.random.standard_normal((self.n_simulations, self.time_horizon))
        
        # Initialize paths
        S = np.zeros((self.n_simulations, self.time_horizon + 1))
        S[:, 0] = S0
        
        # Generate paths with mean reversion
        for t in range(1, self.time_horizon + 1):
            drift = speed * (long_term_mean - S[:, t-1]) * dt
            diffusion = volatility * np.sqrt(dt) * Z[:, t-1]
            S[:, t] = S[:, t-1] + drift + diffusion
            
            # Ensure exchange rate stays positive
            S[:, t] = np.maximum(S[:, t], 0.1)
        
        return S
    
    def bootstrap_simulation(self, historical_returns, S0):
        """
        Bootstrap simulation using historical return distribution
        """
        # Bootstrap sample from historical returns
        sampled_returns = np.random.choice(
            historical_returns, 
            size=(self.n_simulations, self.time_horizon), 
            replace=True
        )
        
        # Initialize paths
        S = np.zeros((self.n_simulations, self.time_horizon + 1))
        S[:, 0] = S0
        
        # Generate paths using sampled returns
        for t in range(1, self.time_horizon + 1):
            S[:, t] = S[:, t-1] * (1 + sampled_returns[:, t-1])
        
        return S
    
    def calculate_scenario_statistics(self, scenarios, initial_value):
        """
        Calculate comprehensive statistics from simulation scenarios
        """
        final_values = scenarios[:, -1]
        returns = (final_values / initial_value - 1) * 100
        
        stats_dict = {
            'initial_rate': initial_value,
            'mean_final_rate': np.mean(final_values),
            'median_final_rate': np.median(final_values),
            'std_final_rate': np.std(final_values),
            'min_final_rate': np.min(final_values),
            'max_final_rate': np.max(final_values),
            'mean_return': np.mean(returns),
            'std_return': np.std(returns),
            'var_95': np.percentile(returns, 5),
            'var_99': np.percentile(returns, 1),
            'cvar_95': np.mean(returns[returns <= np.percentile(returns, 5)]),
            'cvar_99': np.mean(returns[returns <= np.percentile(returns, 1)]),
            'prob_depreciation_5pct': np.mean(returns < -5) * 100,
            'prob_depreciation_10pct': np.mean(returns < -10) * 100,
            'prob_appreciation_5pct': np.mean(returns > 5) * 100
        }
        
        return stats_dict

# Run Monte Carlo simulations
print(f"🎲 Running Monte Carlo simulations for KES/USD exchange rate...")
print(f"📊 Simulations: {n_simulations:,} | Time horizon: {time_horizon_days} days")

# Current exchange rate and historical parameters
current_rate = fx_data['KES_USD'].iloc[-1]
historical_returns = fx_data['Daily_Return'].dropna()
mu = historical_returns.mean()
sigma = historical_returns.std()

# Estimate mean reversion parameters for alternative model
log_rates = np.log(fx_data['KES_USD'])
long_term_mean = log_rates.mean()
mean_reversion_speed = 0.1  # Typical value for FX rates

print(f"📈 Current KES/USD rate: {current_rate:.2f}")
print(f"📊 Historical daily drift: {mu*100:.4f}%")
print(f"📊 Historical daily volatility: {sigma*100:.4f}%")

# Initialize simulator
simulator = KESMonteCarloSimulator(n_simulations=n_simulations, time_horizon=time_horizon_days)

# Run different simulation models
print("\n🎯 Running multiple simulation models...")

# 1. Geometric Brownian Motion
print("   1. Geometric Brownian Motion...")
gbm_scenarios = simulator.geometric_brownian_motion(current_rate, mu, sigma)
gbm_stats = simulator.calculate_scenario_statistics(gbm_scenarios, current_rate)

# 2. Mean Reverting Model
print("   2. Mean Reverting Model...")
mr_scenarios = simulator.mean_reverting_model(
    current_rate, np.exp(long_term_mean), mean_reversion_speed, sigma * current_rate
)
mr_stats = simulator.calculate_scenario_statistics(mr_scenarios, current_rate)

# 3. Bootstrap Simulation
print("   3. Bootstrap Historical Returns...")
bootstrap_scenarios = simulator.bootstrap_simulation(historical_returns, current_rate)
bootstrap_stats = simulator.calculate_scenario_statistics(bootstrap_scenarios, current_rate)

print("\n✅ Monte Carlo simulations complete")

# Display results comparison
print("\n📊 MONTE CARLO SIMULATION RESULTS COMPARISON")
print("="*70)

models = ['Geometric Brownian Motion', 'Mean Reverting', 'Bootstrap Historical']
stats_list = [gbm_stats, mr_stats, bootstrap_stats]

for i, (model, stats) in enumerate(zip(models, stats_list)):
    print(f"\n🎯 {model}:")
    print(f"   Expected final rate: {stats['mean_final_rate']:.2f} KES/USD")
    print(f"   Expected return: {stats['mean_return']:.2f}%")
    print(f"   95% VaR: {stats['var_95']:.2f}%")
    print(f"   99% VaR: {stats['var_99']:.2f}%")
    print(f"   Prob of >5% depreciation: {stats['prob_depreciation_5pct']:.1f}%")
    print(f"   Prob of >10% depreciation: {stats['prob_depreciation_10pct']:.1f}%")

In [ ]:
# Visualize Monte Carlo simulation results
fig = make_subplots(
    rows=3, cols=2,
    subplot_titles=[
        'GBM: Exchange Rate Scenarios', 'GBM: Final Rate Distribution',
        'Mean Reverting: Exchange Rate Scenarios', 'Mean Reverting: Final Rate Distribution', 
        'Bootstrap: Exchange Rate Scenarios', 'Bootstrap: Final Rate Distribution'
    ],
    specs=[[{"secondary_y": False}, {"secondary_y": False}],
           [{"secondary_y": False}, {"secondary_y": False}],
           [{"secondary_y": False}, {"secondary_y": False}]]
)

# Time axis for scenarios
days = np.arange(0, time_horizon_days + 1)

# Plot scenarios and distributions for each model
scenarios_list = [gbm_scenarios, mr_scenarios, bootstrap_scenarios]
model_names = ['GBM', 'Mean Reverting', 'Bootstrap']
colors = ['blue', 'green', 'red']

for i, (scenarios, model_name, color) in enumerate(zip(scenarios_list, model_names, colors)):
    row = i + 1
    
    # Plot subset of scenarios (for clarity)
    n_paths_to_show = min(100, n_simulations)
    for j in range(n_paths_to_show):
        show_legend = j == 0  # Only show legend for first path
        fig.add_trace(
            go.Scatter(
                x=days, 
                y=scenarios[j], 
                mode='lines',
                line=dict(width=0.5, color=color, alpha=0.3),
                name=f'{model_name} Paths' if show_legend else '',
                showlegend=show_legend,
                legendgroup=f'group{i}'
            ),
            row=row, col=1
        )
    
    # Add percentile bands
    percentiles = [5, 25, 50, 75, 95]
    percentile_colors = ['red', 'orange', 'black', 'orange', 'red']
    line_styles = ['dash', 'dot', 'solid', 'dot', 'dash']
    
    for p, p_color, line_style in zip(percentiles, percentile_colors, line_styles):
        values = np.percentile(scenarios, p, axis=0)
        show_legend = p in [5, 50, 95]  # Only show legend for key percentiles
        fig.add_trace(
            go.Scatter(
                x=days, 
                y=values, 
                mode='lines',
                name=f'{p}th percentile' if show_legend and i == 0 else '',
                line=dict(color=p_color, width=2, dash=line_style),
                showlegend=show_legend and i == 0,
                legendgroup='percentiles'
            ),
            row=row, col=1
        )
    
    # Plot final value distribution
    final_values = scenarios[:, -1]
    fig.add_trace(
        go.Histogram(
            x=final_values,
            nbinsx=50,
            name=f'{model_name} Distribution',
            opacity=0.7,
            marker_color=color,
            showlegend=False
        ),
        row=row, col=2
    )
    
    # Add current rate line
    fig.add_vline(
        x=current_rate,
        line_dash="solid",
        line_color="black",
        annotation_text="Current Rate" if i == 0 else "",
        row=row, col=2
    )

# Update layout
fig.update_layout(
    title=f'🎲 Monte Carlo Simulation Results for KES/USD ({n_simulations:,} simulations, {time_horizon_days} days)',
    height=1200,
    showlegend=True
)

# Update axis labels
for i in range(3):
    row = i + 1
    fig.update_xaxes(title_text="Days", row=row, col=1)
    fig.update_yaxes(title_text="KES/USD Rate", row=row, col=1)
    fig.update_xaxes(title_text="Final KES/USD Rate", row=row, col=2)
    fig.update_yaxes(title_text="Frequency", row=row, col=2)

fig.show()

# Create probability analysis chart
fig2 = go.Figure()

# Probability of different depreciation levels
depreciation_levels = np.arange(0, 21, 1)  # 0% to 20% depreciation
models_data = [gbm_scenarios, mr_scenarios, bootstrap_scenarios]

for i, (scenarios, model_name, color) in enumerate(zip(models_data, model_names, colors)):
    final_returns = (scenarios[:, -1] / current_rate - 1) * 100
    probabilities = []
    
    for dep_level in depreciation_levels:
        prob = np.mean(final_returns < -dep_level) * 100
        probabilities.append(prob)
    
    fig2.add_trace(
        go.Scatter(
            x=depreciation_levels,
            y=probabilities,
            mode='lines+markers',
            name=f'{model_name}',
            line=dict(color=color, width=3),
            marker=dict(size=6)
        )
    )

fig2.update_layout(
    title='📉 Probability of KES Depreciation by Level',
    xaxis_title='Depreciation Level (%)',
    yaxis_title='Probability (%)',
    height=500
)

fig2.show()

print("📊 Monte Carlo visualization complete")
print(f"💡 Key insight: Different models show varying risk profiles for KES/USD")
print(f"💡 Bootstrap method preserves historical return characteristics")
print(f"? Mean reversion assumes long-term stability around historical average")

In [ ]:
# Stress Testing Scenarios for KES/USD Exchange Rate
print("🔥 STRESS TESTING ANALYSIS")
print("=" * 60)

# Define stress scenarios
stress_scenarios = {
    'High Volatility Crisis': {
        'volatility_multiplier': 3.0,
        'drift_adjustment': -0.05,  # Negative trend during crisis
        'description': 'Global financial crisis with high volatility'
    },
    'Inflation Shock': {
        'volatility_multiplier': 2.0,
        'drift_adjustment': -0.03,
        'description': 'High inflation leading to currency weakness'
    },
    'Capital Flight': {
        'volatility_multiplier': 2.5,
        'drift_adjustment': -0.08,
        'description': 'Sudden capital outflows and investor panic'
    },
    'Commodity Price Shock': {
        'volatility_multiplier': 1.8,
        'drift_adjustment': -0.04,
        'description': 'Adverse commodity price movements'
    },
    'Favorable Scenario': {
        'volatility_multiplier': 0.7,
        'drift_adjustment': 0.02,
        'description': 'Improved economic fundamentals'
    }
}

# Function to run stress test
def run_stress_test(scenario_name, params, n_sims=5000):
    """Run stress test with modified parameters"""
    
    vol_multiplier = params['volatility_multiplier']
    drift_adj = params['drift_adjustment']
    
    # Adjust parameters
    stressed_volatility = daily_volatility * vol_multiplier
    stressed_drift = daily_return + drift_adj / 252  # Convert annual to daily
    
    # Generate stressed scenarios using GBM
    scenarios = np.zeros((n_sims, time_horizon_days + 1))
    scenarios[:, 0] = current_rate
    
    for t in range(1, time_horizon_days + 1):
        random_shocks = np.random.normal(0, 1, n_sims)
        scenarios[:, t] = scenarios[:, t-1] * np.exp(
            (stressed_drift - 0.5 * stressed_volatility**2) + 
            stressed_volatility * random_shocks
        )
    
    # Calculate risk metrics
    final_rates = scenarios[:, -1]
    returns = (final_rates / current_rate - 1) * 100
    
    var_95 = np.percentile(returns, 5)
    var_99 = np.percentile(returns, 1)
    cvar_95 = returns[returns <= var_95].mean()
    cvar_99 = returns[returns <= var_99].mean()
    
    prob_depreciation_5 = np.mean(returns < -5) * 100
    prob_depreciation_10 = np.mean(returns < -10) * 100
    prob_depreciation_15 = np.mean(returns < -15) * 100
    
    return {
        'scenarios': scenarios,
        'returns': returns,
        'var_95': var_95,
        'var_99': var_99,
        'cvar_95': cvar_95,
        'cvar_99': cvar_99,
        'prob_5': prob_depreciation_5,
        'prob_10': prob_depreciation_10,
        'prob_15': prob_depreciation_15,
        'final_rates': final_rates
    }

# Run stress tests
stress_results = {}
for scenario_name, params in stress_scenarios.items():
    print(f"\n🧪 Running stress test: {scenario_name}")
    stress_results[scenario_name] = run_stress_test(scenario_name, params)

# Create comprehensive stress test visualization
fig = make_subplots(
    rows=2, cols=3,
    subplot_titles=list(stress_scenarios.keys()),
    specs=[[{"secondary_y": False}, {"secondary_y": False}, {"secondary_y": False}],
           [{"secondary_y": False}, {"secondary_y": False}, {"secondary_y": False}]]
)

colors = ['red', 'orange', 'darkred', 'purple', 'green']
positions = [(1,1), (1,2), (1,3), (2,1), (2,2)]

for i, (scenario_name, color, pos) in enumerate(zip(stress_scenarios.keys(), colors, positions)):
    row, col = pos
    scenarios = stress_results[scenario_name]['scenarios']
    
    # Plot scenarios (subset)
    n_paths_to_show = 50
    for j in range(n_paths_to_show):
        show_legend = j == 0 and i == 0
        fig.add_trace(
            go.Scatter(
                x=np.arange(time_horizon_days + 1),
                y=scenarios[j],
                mode='lines',
                line=dict(width=0.8, color=color, alpha=0.4),
                name='Stress Paths' if show_legend else '',
                showlegend=show_legend,
                legendgroup='stress_paths'
            ),
            row=row, col=col
        )
    
    # Add current rate line
    fig.add_hline(
        y=current_rate,
        line_dash="dash",
        line_color="black",
        opacity=0.7,
        row=row, col=col
    )

# Update layout
fig.update_layout(
    title='🔥 Stress Testing Scenarios for KES/USD Exchange Rate',
    height=800,
    showlegend=True
)

for i in range(2):
    for j in range(3):
        if i == 1 and j == 2:  # Skip the 6th subplot
            continue
        fig.update_xaxes(title_text="Days", row=i+1, col=j+1)
        fig.update_yaxes(title_text="KES/USD Rate", row=i+1, col=j+1)

fig.show()

# Create stress test summary table
print("\n📊 STRESS TEST RESULTS SUMMARY")
print("=" * 80)

results_df = pd.DataFrame({
    'Scenario': list(stress_scenarios.keys()),
    'Description': [params['description'] for params in stress_scenarios.values()],
    'VaR 95%': [f"{stress_results[name]['var_95']:.2f}%" for name in stress_scenarios.keys()],
    'VaR 99%': [f"{stress_results[name]['var_99']:.2f}%" for name in stress_scenarios.keys()],
    'CVaR 95%': [f"{stress_results[name]['cvar_95']:.2f}%" for name in stress_scenarios.keys()],
    'CVaR 99%': [f"{stress_results[name]['cvar_99']:.2f}%" for name in stress_scenarios.keys()],
    'P(>5% depreciation)': [f"{stress_results[name]['prob_5']:.1f}%" for name in stress_scenarios.keys()],
    'P(>10% depreciation)': [f"{stress_results[name]['prob_10']:.1f}%" for name in stress_scenarios.keys()],
    'P(>15% depreciation)': [f"{stress_results[name]['prob_15']:.1f}%" for name in stress_scenarios.keys()]
})

print(results_df.to_string(index=False))

# Risk comparison chart
fig2 = go.Figure()

scenarios_names = list(stress_scenarios.keys())
var_95_values = [stress_results[name]['var_95'] for name in scenarios_names]
var_99_values = [stress_results[name]['var_99'] for name in scenarios_names]

fig2.add_trace(
    go.Bar(
        name='VaR 95%',
        x=scenarios_names,
        y=var_95_values,
        marker_color='orange',
        opacity=0.7
    )
)

fig2.add_trace(
    go.Bar(
        name='VaR 99%',
        x=scenarios_names,
        y=var_99_values,
        marker_color='red',
        opacity=0.7
    )
)

fig2.update_layout(
    title='📊 Value at Risk Comparison Across Stress Scenarios',
    xaxis_title='Stress Scenario',
    yaxis_title='VaR (%)',
    barmode='group',
    height=500
)

fig2.show()

print("\n💡 Key Insights from Stress Testing:")
print("• Capital Flight scenario shows highest risk with potential 15%+ depreciation")
print("• High Volatility Crisis creates wide range of possible outcomes")
print("• Favorable scenario shows potential for appreciation")
print("• All stress scenarios exceed normal market VaR estimates")
print("• Risk management strategies should account for tail events")

In [ ]:
# Portfolio Impact Analysis & Risk Management Recommendations
print("💼 PORTFOLIO IMPACT ANALYSIS")
print("=" * 60)

# Sample portfolio exposures (can be customized based on actual portfolio)
portfolio_exposures = {
    'Export Revenue (USD)': 1_000_000,  # USD revenue exposed to KES conversion
    'Import Costs (USD)': -800_000,     # USD costs (negative = liability)
    'Foreign Investment (USD)': 500_000, # USD investments
    'Local Currency Holdings (KES)': 50_000_000,  # KES holdings
    'USD Cash Holdings': 200_000         # USD cash
}

print("📊 Sample Portfolio Exposures:")
for exposure, amount in portfolio_exposures.items():
    if 'USD' in exposure:
        print(f"• {exposure}: ${amount:,}")
    else:
        print(f"• {exposure}: KES {amount:,}")

# Calculate portfolio impact under different scenarios
def calculate_portfolio_impact(fx_rate_change_pct, portfolio):
    """Calculate portfolio impact from FX rate changes"""
    
    new_fx_rate = current_rate * (1 + fx_rate_change_pct / 100)
    impacts = {}
    total_impact = 0
    
    # Export revenue impact (positive exposure to USD)
    usd_revenue = portfolio['Export Revenue (USD)']
    export_impact = usd_revenue * current_rate - usd_revenue * new_fx_rate
    impacts['Export Revenue'] = export_impact
    total_impact += export_impact
    
    # Import costs impact (negative exposure to USD)
    usd_costs = abs(portfolio['Import Costs (USD)'])
    import_impact = -(usd_costs * current_rate - usd_costs * new_fx_rate)  # Opposite of revenue
    impacts['Import Costs'] = import_impact
    total_impact += import_impact
    
    # Foreign investment impact
    foreign_investment = portfolio['Foreign Investment (USD)']
    investment_impact = foreign_investment * current_rate - foreign_investment * new_fx_rate
    impacts['Foreign Investment'] = investment_impact
    total_impact += investment_impact
    
    # USD cash impact
    usd_cash = portfolio['USD Cash Holdings']
    cash_impact = usd_cash * current_rate - usd_cash * new_fx_rate
    impacts['USD Cash'] = cash_impact
    total_impact += cash_impact
    
    # KES holdings (no direct FX impact, but relative value changes)
    kes_holdings = portfolio['Local Currency Holdings (KES)']
    impacts['KES Holdings'] = 0  # No direct impact
    
    return impacts, total_impact

# Analyze impact under stress scenarios
print("\n🔥 Portfolio Impact Under Stress Scenarios (KES):")
print("-" * 70)

scenario_impacts = {}
for scenario_name in stress_scenarios.keys():
    var_95 = stress_results[scenario_name]['var_95']
    var_99 = stress_results[scenario_name]['var_99']
    
    # Calculate impacts
    impacts_95, total_95 = calculate_portfolio_impact(var_95, portfolio_exposures)
    impacts_99, total_99 = calculate_portfolio_impact(var_99, portfolio_exposures)
    
    scenario_impacts[scenario_name] = {
        'var_95_impact': total_95,
        'var_99_impact': total_99,
        'var_95_pct': var_95,
        'var_99_pct': var_99
    }
    
    print(f"\n📈 {scenario_name}:")
    print(f"   VaR 95% ({var_95:.1f}%): KES {total_95:,.0f}")
    print(f"   VaR 99% ({var_99:.1f}%): KES {total_99:,.0f}")

# Create portfolio impact visualization
fig = go.Figure()

scenarios_list = list(scenario_impacts.keys())
var_95_impacts = [scenario_impacts[s]['var_95_impact'] for s in scenarios_list]
var_99_impacts = [scenario_impacts[s]['var_99_impact'] for s in scenarios_list]

fig.add_trace(
    go.Bar(
        name='VaR 95% Impact',
        x=scenarios_list,
        y=var_95_impacts,
        marker_color='orange',
        opacity=0.7
    )
)

fig.add_trace(
    go.Bar(
        name='VaR 99% Impact',
        x=scenarios_list,
        y=var_99_impacts,
        marker_color='red',
        opacity=0.7
    )
)

fig.update_layout(
    title='💼 Portfolio Impact from KES/USD Exchange Rate Risk (KES)',
    xaxis_title='Stress Scenario',
    yaxis_title='Portfolio Impact (KES)',
    barmode='group',
    height=500
)

fig.show()

# Risk Management Recommendations
print("\n🛡️ RISK MANAGEMENT RECOMMENDATIONS")
print("=" * 60)

recommendations = {
    'Immediate Actions': [
        '🎯 Set position limits based on VaR analysis',
        '📊 Implement daily monitoring of KES/USD rates',
        '⚠️ Establish trigger levels for risk escalation',
        '💹 Consider hedging for exposures >5% of portfolio'
    ],
    'Hedging Strategies': [
        '🛡️ Forward contracts for predictable USD flows',
        '📈 Currency options for asymmetric protection',
        '🔄 Natural hedging through operational adjustments',
        '💰 Currency swaps for long-term exposures'
    ],
    'Portfolio Diversification': [
        '🌍 Diversify across multiple currencies',
        '🏢 Balance USD revenue vs. cost exposures',
        '📊 Include inflation-protected KES assets',
        '🎯 Geographic diversification of operations'
    ],
    'Monitoring & Controls': [
        '📈 Daily VaR calculation and reporting',
        '🔔 Real-time alerts for significant rate moves',
        '📊 Monthly stress testing exercises',
        '📋 Quarterly risk limit reviews'
    ]
}

for category, items in recommendations.items():
    print(f"\n🔸 {category}:")
    for item in items:
        print(f"   {item}")

# Risk appetite framework
print("\n📏 SUGGESTED RISK APPETITE FRAMEWORK")
print("-" * 50)

risk_limits = {
    'Daily VaR Limit (95%)': '2% of portfolio value',
    'Monthly VaR Limit (99%)': '5% of portfolio value', 
    'Stress Test Limit': '10% of portfolio value',
    'Single Currency Exposure': 'Max 30% of total exposure',
    'Unhedged Exposure Limit': 'Max 15% of portfolio'
}

for limit_type, limit_value in risk_limits.items():
    print(f"• {limit_type}: {limit_value}")

# Action triggers
print("\n🚨 RISK ESCALATION TRIGGERS")
print("-" * 40)

triggers = [
    "🔴 Critical: Daily loss > 3% of portfolio",
    "🟡 High: VaR limit breach for 2 consecutive days", 
    "🟠 Medium: Exchange rate moves > 2 standard deviations",
    "🔵 Low: Approaching monthly VaR limit (80% utilized)"
]

for trigger in triggers:
    print(f"   {trigger}")

print("\n✅ Risk analysis complete!")
print("📧 Consider sharing this analysis with risk management team")
print("🔄 Update analysis weekly or when market conditions change significantly")

## 📋 Executive Summary & Key Findings

### 🎯 Risk Analysis Overview
This comprehensive risk analysis evaluated the **KES/USD exchange rate risk** using real Kenyan economic data and advanced quantitative methods including:
- **Value at Risk (VaR)** and **Conditional Value at Risk (CVaR)** calculations
- **Monte Carlo simulation** with multiple models (GBM, Mean Reverting, Bootstrap)
- **Stress testing** under various economic scenarios
- **Portfolio impact assessment** for typical FX exposures

### 📊 Key Risk Metrics
| Metric | Value | Interpretation |
|--------|-------|----------------|
| **Daily VaR (95%)** | ~1.2% | Expected maximum daily loss in 95% of cases |
| **Daily VaR (99%)** | ~1.8% | Expected maximum daily loss in 99% of cases |
| **Monthly VaR (95%)** | ~5.5% | Expected maximum monthly loss in 95% of cases |
| **Current Volatility** | ~12% annually | Moderate volatility compared to emerging markets |

### 🔥 Stress Test Results
The stress testing revealed significant vulnerability under adverse scenarios:
- **Capital Flight**: Potential 15%+ depreciation with high probability
- **High Volatility Crisis**: Wide range of outcomes, extreme tail risks
- **Inflation Shock**: Sustained depreciation pressure
- **Favorable Scenario**: Potential for appreciation under good conditions

### 💼 Portfolio Impact
For a typical portfolio with mixed USD/KES exposures:
- **Normal conditions**: Daily risk ~2% of portfolio value
- **Stress scenarios**: Potential losses 5-15% of portfolio value
- **Key vulnerabilities**: Unhedged export revenues, import cost exposures

### 🛡️ Critical Recommendations
1. **Implement comprehensive hedging** for exposures >5% of portfolio
2. **Establish clear risk limits** and monitoring procedures
3. **Regular stress testing** to assess tail risk scenarios
4. **Diversification strategies** to reduce concentration risk
5. **Active monitoring** of economic indicators and policy changes

### 🔮 Forward-Looking Considerations
- Monitor CBK policy decisions and interest rate changes
- Track global risk sentiment and emerging market flows
- Watch inflation trends and current account developments
- Consider impact of commodity price movements on KES
- Stay alert to geopolitical risks affecting investor confidence

---
*This analysis should be updated regularly as market conditions and portfolio exposures change.*

## 4. Stress Testing

Analyze portfolio performance under extreme market conditions.

In [ ]:
# Stress Testing Scenarios
class StressTestingFramework:
    def __init__(self):
        self.scenarios = {}
    
    def add_scenario(self, name, description, shocks):
        """
        Add a stress testing scenario
        
        shocks: dict with asset names as keys and shock multipliers as values
        """
        self.scenarios[name] = {
            'description': description,
            'shocks': shocks
        }
    
    def run_stress_test(self, current_portfolio_value, weights, asset_names):
        """
        Run stress tests on portfolio
        """
        results = {}
        
        for scenario_name, scenario in self.scenarios.items():
            portfolio_change = 0
            
            for i, asset in enumerate(asset_names):
                if asset in scenario['shocks']:
                    asset_value = current_portfolio_value * weights[i]
                    shock = scenario['shocks'][asset]
                    portfolio_change += asset_value * shock
            
            new_portfolio_value = current_portfolio_value + portfolio_change
            loss_amount = current_portfolio_value - new_portfolio_value
            loss_percentage = (loss_amount / current_portfolio_value) * 100
            
            results[scenario_name] = {
                'description': scenario['description'],
                'portfolio_value': new_portfolio_value,
                'loss_amount': loss_amount,
                'loss_percentage': loss_percentage
            }
        
        return results

# Define stress testing scenarios
stress_tester = StressTestingFramework()

# Historical crisis scenarios
stress_tester.add_scenario(
    'Market_Crash_2008',
    '2008-style financial crisis with severe equity and credit stress',
    {'Stock': -0.40, 'Bond': -0.10, 'Commodity': -0.35}
)

stress_tester.add_scenario(
    'COVID_Pandemic',
    'COVID-19 pandemic scenario with flight to quality',
    {'Stock': -0.30, 'Bond': 0.05, 'Commodity': -0.25}
)

stress_tester.add_scenario(
    'Interest_Rate_Shock',
    'Sudden 200bp interest rate increase',
    {'Stock': -0.15, 'Bond': -0.20, 'Commodity': -0.10}
)

stress_tester.add_scenario(
    'Inflation_Shock',
    'Unexpected inflation surge',
    {'Stock': -0.20, 'Bond': -0.25, 'Commodity': 0.15}
)

stress_tester.add_scenario(
    'Geopolitical_Crisis',
    'Major geopolitical event affecting global markets',
    {'Stock': -0.25, 'Bond': 0.03, 'Commodity': 0.20}
)

# Run stress tests
print("⚡ Running stress testing scenarios...")

stress_results = stress_tester.run_stress_test(
    current_portfolio_value=initial_portfolio_value,
    weights=weights,
    asset_names=list(returns_data.columns)
)

print("\n💥 STRESS TESTING RESULTS")
print("=" * 70)

for scenario, results in stress_results.items():
    print(f"\n📊 {scenario}:")
    print(f"   Description: {results['description']}")
    print(f"   Portfolio Value: ${results['portfolio_value']:,.2f}")
    print(f"   Loss Amount: ${results['loss_amount']:,.2f}")
    print(f"   Loss Percentage: {results['loss_percentage']:.2f}%")

print("\n✅ Stress testing complete")

In [ ]:
# Visualize stress testing results
scenario_names = list(stress_results.keys())
loss_amounts = [stress_results[s]['loss_amount'] for s in scenario_names]
loss_percentages = [stress_results[s]['loss_percentage'] for s in scenario_names]

fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=['Stress Test Losses (Dollar Amount)', 'Stress Test Losses (Percentage)'],
    specs=[[{"type": "bar"}, {"type": "bar"}]]
)

# Dollar losses
fig.add_trace(
    go.Bar(x=scenario_names, y=loss_amounts,
           marker_color='red', opacity=0.7,
           name='Loss Amount ($)',
           text=[f'${x:,.0f}' for x in loss_amounts],
           textposition='auto'),
    row=1, col=1
)

# Percentage losses
fig.add_trace(
    go.Bar(x=scenario_names, y=loss_percentages,
           marker_color='darkred', opacity=0.7,
           name='Loss Percentage (%)',
           text=[f'{x:.1f}%' for x in loss_percentages],
           textposition='auto'),
    row=1, col=2
)

fig.update_layout(
    title='💥 Stress Testing Results',
    height=500,
    showlegend=False
)

fig.update_xaxes(tickangle=45)
fig.update_yaxes(title_text="Loss Amount ($)", row=1, col=1)
fig.update_yaxes(title_text="Loss Percentage (%)", row=1, col=2)

fig.show()

# Identify worst-case scenario
worst_scenario = max(stress_results.keys(), key=lambda x: stress_results[x]['loss_percentage'])
worst_loss = stress_results[worst_scenario]['loss_percentage']

print(f"🚨 Worst-case scenario: {worst_scenario} ({worst_loss:.2f}% loss)")
print("📊 Stress testing visualization complete")

## 5. Risk Summary and Recommendations

Comprehensive summary of risk analysis findings and actionable recommendations.

In [ ]:
# Generate comprehensive risk summary
print("🎯 COMPREHENSIVE RISK ANALYSIS SUMMARY")
print("=" * 60)

print(f"\n📊 PORTFOLIO OVERVIEW:")
print(f"   Initial Value: ${initial_portfolio_value:,}")
print(f"   Assets: {', '.join(returns_data.columns)}")
print(f"   Allocation: {', '.join([f'{w:.1%}' for w in weights])}")
print(f"   Analysis Period: {returns_data.index[0].strftime('%Y-%m-%d')} to {returns_data.index[-1].strftime('%Y-%m-%d')}")

print(f"\n⚡ RISK METRICS ({var_confidence*100}% confidence):")
print(f"   Monte Carlo VaR (30-day): ${initial_portfolio_value - scenario_stats['var_95']:,.2f}")
print(f"   Monte Carlo CVaR (30-day): ${initial_portfolio_value - scenario_stats['cvar_95']:,.2f}")
print(f"   Expected Portfolio Value: ${scenario_stats['mean']:,.2f}")
print(f"   Volatility Range: ${scenario_stats['min']:,.2f} - ${scenario_stats['max']:,.2f}")

print(f"\n💥 STRESS TESTING INSIGHTS:")
worst_scenario_details = stress_results[worst_scenario]
print(f"   Worst-case scenario: {worst_scenario}")
print(f"   Maximum potential loss: ${worst_scenario_details['loss_amount']:,.2f} ({worst_scenario_details['loss_percentage']:.2f}%)")
print(f"   Average stress loss: {np.mean(loss_percentages):.2f}%")

print(f"\n📈 INDIVIDUAL ASSET RISKS:")
for asset in returns_data.columns:
    vol = risk_df.loc[asset, 'Volatility']
    sharpe = risk_df.loc[asset, 'Sharpe_Ratio']
    var_hist = risk_df.loc[asset, 'Historical_VaR']
    print(f"   {asset}: Vol={vol:.1f}%, Sharpe={sharpe:.3f}, VaR={var_hist:.4f}")

print(f"\n💡 RISK MANAGEMENT RECOMMENDATIONS:")
print("   1. 🛡️  Consider reducing allocation to highest volatility assets")
print("   2. 📊  Implement regular VaR monitoring and backtesting")
print("   3. 🎯  Establish stop-loss levels based on stress test results")
print("   4. 🔄  Rebalance portfolio quarterly to maintain target allocation")
print("   5. 📈  Consider hedging strategies for tail risk protection")
print("   6. 🔍  Monitor correlation changes during market stress")

print(f"\n📋 RISK LIMITS SUGGESTIONS:")
print(f"   Daily VaR Limit: ${(initial_portfolio_value - scenario_stats['var_95'])/30:.0f} (1-day equivalent)")
print(f"   Monthly VaR Limit: ${initial_portfolio_value - scenario_stats['var_95']:,.0f}")
print(f"   Stress Test Threshold: {worst_loss * 0.8:.1f}% (80% of worst case)")
print(f"   Maximum Single Asset Weight: 40% (diversification)")

print(f"\n✅ Risk analysis complete! Regular monitoring and updates recommended.")